#### Identificar quais REG values vamos precisar (que estao na base da PaulaTorres):

In [2]:
import pandas as pd

In [ ]:
entrada = pd.read_csv(r'base_entradas_202407142340.csv', low_memory=False)
saida = pd.read_csv(r'base_saidas_202407142342.csv', low_memory=False)

In [5]:
unique_reg_values = set()
unique_reg_values.update(entrada['reg'].dropna().unique())
for value in unique_reg_values:
    print(value)

D100
C500
C100
D500


In [6]:
unique_reg_values = set()
unique_reg_values.update(saida['reg'].dropna().unique())
for value in unique_reg_values:
    print(value)

59
55
65
C190
59


Valores de reg:  
Saida: 59, 55, C190, 65  
Entrada: D100, C100, C500, D500  

### Leitura de tabelas:

In [4]:
import fitz
import re

In [ ]:
import tabula
import pandas as pd
import json
import re

# Remover quebras de linha e corrigir espaçamento
def clean_text(text):
    if isinstance(text, str):
        # Substituir caracteres relacionados a quebras de linha por espaço
        text = re.sub(r'[\r\n]+', ' ', text)
        # Remover espaços múltiplos
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    return text

# Função para processar e limpar as colunas de um DataFrame
def clean_df_columns(df):
    df.columns = [clean_text(col) for col in df.columns]
    return df

# Verificar se a tabela é considerada lixo com base em NaNs e conteúdo
def is_junk_table(df, threshold=0.5):
    total_cells = df.size
    total_na_cells = df.isna().sum().sum()
    na_percentage = total_na_cells / total_cells

    col_na_percentage = df.isna().mean().mean()
    row_na_percentage = df.isna().mean(axis=1).mean()

    return na_percentage > 0.5 or col_na_percentage > threshold or row_na_percentage > threshold

pdf_path = r"C:\Users\gabri\Documents\PJ_Docs\Nutax\Guia Prático EFD - Versão 3.1.6.pdf"
pages = "7-349"

# Ler as tabelas das páginas especificadas
tables = tabula.read_pdf(pdf_path, pages=pages, lattice=True, multiple_tables=True)

cleaned_tables = []

for table in tables:
    if not isinstance(table, pd.DataFrame):
        table = pd.DataFrame(table)
        
    #Limpar tabelas
    table = clean_df_columns(table)
    # Remover colunas em que todas as linhas são NaN
    table = table.dropna(axis=1, how='all')
    # Remover colunas 'Unnamed'
    table = table.loc[:, ~table.columns.str.contains('^Unnamed')]
    # Remover tabelas com todos os valores NaN e tabelas com muitos valores NaN
    if table.empty or is_junk_table(table):
        continue
    # Aplicar clean_text a todos os valores das células
    table = table.applymap(clean_text)
    # Filtrar linhas com excesso de espaços em branco ou valores NaN
    table = table.dropna(thresh=int(0.5*len(table.columns)))
    # Filtrar ainda mais as linhas em que faltam dados significativos
    table = table[~table.apply(lambda row: all(pd.isna(row) or row == '' for row in row), axis=1)]

    cleaned_tables.append(table)

# Função para converter DataFrame em estrutura JSON
def df_to_json(df, table_id):
    return {
        "table_id": table_id,
        "columns": df.columns.tolist(),
        "data": df.fillna("").to_dict(orient='records')
    }

# Converter cada tabela limpa em uma estrutura JSON
json_tables = [df_to_json(table, idx + 1) for idx, table in enumerate(cleaned_tables)]

# Salvar as estruturas JSON em um arquivo
output_json_path = r"C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\Nutax\Modelo IA\Dicionario_Fiscal\cleaned_tables.json"
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(json_tables, f, ensure_ascii=False, indent=4)


In [16]:
import tabula
paginas = "7-349"
tabela_text = tabula.read_pdf(atalho, pages=paginas, lattice=True)

In [17]:
tabela_text

[  Bloco                                          Descrição
 0     0              Abertura, Identificação e Referências
 1  B***                     Escrituração e Apuração do ISS
 2     C      Documentos Fiscais I – Mercadorias (ICMS/IPI)
 3     D            Documentos Fiscais II – Serviços (ICMS)
 4     E                          Apuração do ICMS e do IPI
 5    G*  Controle do Crédito de ICMS do Ativo Permanent...
 6     H                                  Inventário Físico
 7   K**                  Controle da Produção e do Estoque
 8     1                                 Outras Informações
 9     9         Controle e Encerramento do Arquivo Digital,
   Registro                                          Descrição
 0     C105  OPERAÇÕES COM ICMS ST RECOLHIDO PARA UF DIVERS...
 1     1700                      DOCUMENTOS FISCAIS UTILIZADOS
 2     1710         DOCUMENTOS FISCAIS CANCELADOS/INUTILIZADOS
 3     1800  DCTA – DEMONSTRATIVO DE CRÉDITO DO ICMS SOBRE ...,
   Registro  Número do 

In [22]:
print(len(tabela_text))

4754


In [35]:
tabela_text[10]

,Bloco,Unnamed: 0,Descrição,Registro,Nível,Ocorrência,Obrigatoriedade do bloco\r(Todos os contribuintes)
0,K,NaN,Abertura do Bloco K,K001,1,1,O
1,K,NaN,Período de Apuração do ICMS/IPI,K100,2,V,OC
2,K,NaN,Estoque Escriturado,K200,3,1:N,OC
3,K,NaN,Outras Movimentações Internas entre Mercadorias,K220,3,1:N,OC
4,K,NaN,Itens Produzidos,K230,3,1:N,OC
